# Unsupervised ML - Anime Recommendation

A revision of project 1

The dataset used here is the Anime Recommendations Database on Kaggle, which consists of two separate csv files and contains information about 12,294 anime and 73,516 users. The goal here is to recommend at most three unwatched anime to a user based the user's rating history.

In [1]:
# import block
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# import functions
from project1 import top_100_rate_count

In [2]:
# import the first dataset - rating.csv
rating = pd.read_csv('rating.csv', sep=',')
rating['rating'] = rating['rating'].apply(lambda x: np.nan if x==-1 else x)
print('shape of rating is', rating.shape)
rating.head()

shape of rating is (7813737, 3)


,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [3]:
# import the second dataset - anime.csv
anime = pd.read_csv('anime.csv', sep=',').rename(columns = {'name': 'anime_name'})
print('shape of anime is', anime.shape)
anime.head()

shape of anime is (12294, 7)


,anime_id,anime_name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
